## External functions for Fundus of the Eye detecting
Makes the main application clear for user

## Libraries

#### Image processing and general libs

In [2]:
import cv2
import functools
from IPython.display import Markdown, clear_output, display, HTML
from ipywidgets import widgets, Layout,Label, HBox, VBox, Box
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
from skimage.filters import gaussian, threshold_li, sato
from skimage import img_as_float, img_as_ubyte
from skimage.color import rgb2hsv,rgb2gray
import skimage.morphology as mp
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

#### Machine Learning Libraries

In [3]:
from numba import jit, cuda 
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, make_scorer, recall_score, accuracy_score, precision_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_predict

Using TensorFlow backend.


### General functions for displaying and loading pictures

In [4]:
def display_picture(image, title=None, file=None):
    fig = plt.figure(figsize=(6,6))
    sub = fig.add_subplot(111)
    if title is not None:
        sub.set_title(title)
    plt.axis('off')
    plt.imshow(image) 
    if file is not None:
        plt.savefig(file)

def display_results(image_array,titles_array, file="img/image-results/test.jpg"):
    count = len(image_array)
    fig, axs = plt.subplots(nrows=1, ncols=count, figsize=(30,6))
    for i, ax in enumerate(axs.flatten()):
        plt.sca(ax)
        plt.axis('off')
        plt.imshow(images_array[i])
        plt.title(titles_array[i])

    plt.savefig(file)
    plt.show()
    
def display_statistics_plot(expect,mask,tpr,fpr,roc_auc,file="img/image-results/stats/stat-test.jpg"):
    fig = plt.figure(figsize=(20,6))
    title = "Classifications metrics"
    fig.suptitle(title, fontsize=16)

    original = fig.add_subplot(131)
    original.set_title('Expert mask')
    plt.axis('off')
    plt.imshow(expect)

    roc_curve = fig.add_subplot(132)
    roc_curve.set_title('Receiver Operating Characteristic')
    roc_curve.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
    roc_curve.plot([0, 1], [0, 1], color='navy', linestyle='--')
    xlim=[0.0, 1.0]
    ylim=[0.0, 1.05]
    roc_curve.set_xlim(xlim)
    roc_curve.set_ylim(ylim)
    roc_curve.set_xlabel('False Positive Rate')
    roc_curve.set_ylabel('True Positive Rate')

    actual = fig.add_subplot(133)
    actual.set_title('Obtained mask')
    plt.axis('off')
    plt.imshow(mask)
    
    plt.savefig(file)
    plt.show()

def draw_roc_curve(tpr,fpr,roc_auc,title="Receiver Operating Characteristic"):
    plt.title(title)
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

def display_label_distribution(y_train,y_test):
    classes_number = 2
    fig, ax = plt.subplots()
    training_counts = [None] * classes_number 
    testing_counts = [None] * classes_number
    for i in range(classes_number):
        training_counts[i] = len(y_train[y_train == i])
        testing_counts[i] = len(y_test[y_test == i])

    train_bar = plt.bar(np.arange(classes_number)-0.2, training_counts, align='center', color = 'pink', alpha=0.75, width = 0.41, label='Training')
    test_bar = plt.bar(np.arange(classes_number)+0.2, testing_counts, align='center', color = 'teal', alpha=0.75, width = 0.41, label = 'Testing')
   
    ax.set_xlabel('Labels')
    x_ticks_labels = ['Background','Vessel']
    ax.set_xticks((0,1))
    ax.set_xticklabels(x_ticks_labels,rotation='horizontal', fontsize=14)
    ax.set_ylabel('Count')
    plt.title('Label distribution in the training and test set')
    plt.legend(bbox_to_anchor=(1.05, 1), handles=[train_bar, test_bar], loc=2)
    plt.grid(True)
    plt.show()

def get_input_model_params():
    bootstrap=bootstrap_input.value
    n_estimators=n_estimators_input.value
    criterion=criterion_input.value
    max_depth=max_depth_input.value
    max_features=max_features_input.value
    min_samples_leaf=min_samples_leaf_input.value
    min_samples_split=min_samples_split_input.value
    return bootstrap,n_estimators,criterion,max_depth,max_features,min_samples_leaf,min_samples_split

### Statistics section

In [5]:
def apply_pic_on_pic(img1,img2):
    img2=img_as_float(rgb2gray(img2))
    h = img1.shape[0]
    w = img2.shape[1]

    for y in range(h):
        for x in range(w):
            img1[y, x] = 255 if img2[y, x] > 0 else img1[y,x]

    return img1

def calculate_statistics(result_masks, expected_array, filename="test"):
    accuracy_list=[]
    sensitivity_list=[]
    precision_list=[]
    specificity_list=[]
    fpr_list=[]
    f_score_list=[]
    g_mean_list=[]
    
    for i,(mask, expect) in enumerate(zip(result_masks, expected_array)):
        mask_copy, expect_copy = binarize_images(mask.copy(),expect.copy())
        mask_array = mask_copy.flatten()
        expect_array = expect_copy.flatten()
        
        true_negative, false_positive, false_negative, true_positive = confusion_matrix(mask_array, expect_array, labels=[0,1]).ravel()
        fpr, tpr, _ = roc_curve(expect_array, mask_array)
        roc_auc = auc(fpr, tpr)
        
        accuracy =  (true_positive + true_negative) / (true_positive+true_negative+false_positive+false_negative)#trafnosc
        sensitivity =  true_positive / (true_positive + false_negative)#czulosc : tp / (tp + fn)
        precision = true_positive / (true_positive + false_positive) # tp / tp + fp
        specificity = true_negative / (true_negative + false_positive) #swoistosc: tn / (tn + fp)
        f_score = 2 * precision * sensitivity /  ( precision + sensitivity)
        g_mean = np.sqrt(sensitivity*specificity)
        
        accuracy_list.append(accuracy)
        sensitivity_list.append(sensitivity)
        precision_list.append(precision)
        specificity_list.append(specificity)
        f_score_list.append(f_score) 
        g_mean_list.append(g_mean)
        
        filepath="img/image-results/stats/stat-img"+filename+str(i)+".jpg"
        display_statistics_plot(expect,mask,tpr,fpr,roc_auc,filepath)
        
    return accuracy_list,sensitivity_list,precision_list,specificity_list,f_score_list,g_mean_list

def binarize_images(mask,expect,value=[1,1,1],thresh=0):
    black=[0,0,0]
    h = expect.shape[0]
    w = expect.shape[1]
    for y in range(h):
        for x in range(w):
            expect_vessel = expect[y,x]
            detect_vessel = mask[y,x]
            if np.amax(expect_vessel) > thresh:
                expect[y,x]=value
                if np.amax(detect_vessel) >thresh:
                    mask[y,x]=value
                else:
                    mask[y,x]=black
            else:
                expect[y,x]=black
                if np.amax(detect_vessel) > thresh:
                    mask[y,x]=value
                else:
                    mask[y,x]=black
    return mask,expect

### Class for uploading images for further processing

In [6]:
class ImageReader:
    
    def __init__(self):
        self._images_array = []
        self._expected_array = []
        self._official_mask=[]

    def load_image(self,path,file):
        if('.jpg' in file or '.png' or '.tif' in file):
            full_file = path+file
            img = cv2.imread(full_file)
            return img
    
    def upload_single_image(self, filename):
        path = 'img/'
        self._images_array.clear()
        img = self.load_image(path,filename)
        
        self._images_array.append(img)
        
        expected_path = 'img/expected/'
        self._expected_array.clear()
        expected = self.load_image(expected_path,filename)
        self._expected_array.append(expected)
        
        self._official_mask.clear()
        mask_path = 'img/mask/'
        filename = '01_h_mask.tif'
        self._official_mask.append(self.load_image(mask_path,filename))

    
    def upload_many_images(self, path):
        self._images_array.clear()
        
        expected_path = path+'expected/'
        self._expected_array.clear()
        
        files = [f for f in listdir(path) if isfile(join(path, f))]
        for i,file in enumerate(files):
            img = self.load_image(path,file)
            
            self._images_array.append(img)
            
            exp = self.load_image(expected_path,file)
            self._expected_array.append(exp)
        
        self._official_mask.clear()
        mask_path = 'img/mask/'
        filename = '01_h_mask.tif'
        self._official_mask.append(self.load_image(mask_path,filename))
        print(f'Updated {i+1} files!')

    def get_images_array(self):
        return self._images_array
    
    def get_expected_array(self):
        return self._expected_array
    
    def get_official_mask(self):
        return self._official_mask

### Class for manual fundus detection

In [7]:
class ImageProcessor:
    
    def __init__(self, imageReader):
        self._images_array = imageReader.get_images_array()
        self._expected_array = imageReader.get_expected_array()
        self._official_mask = imageReader.get_official_mask()
        
        self._result_masks = []
        self._image_progresses = []
        self._result_images = []
    
    def whole_image_processing(self):
        self.find_fundus()
        self.count_statistics()
    
    def find_fundus(self):
        if len(self._images_array)==0:
            display (Markdown('<span style="color: #ff0000">Upload image first!</span>'))
            return
    
        for i,img in enumerate(self._images_array):
            img_orig = img.copy()
            img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
            
            initial = self.initial_processing(img)
            edges = self.edge_detecting(initial)
            detections_mask = self.vessels_mask_creating(edges)
            final_fundus = self.fundus_final_processing(detections_mask)

            final_fundus = cv2.cvtColor(final_fundus, cv2.COLOR_BGR2RGB)
            self._result_masks.append(final_fundus)

            detection_on_orygin = apply_pic_on_pic(img_orig.copy(),final_fundus)
            self._result_images.append(detection_on_orygin)

            image_progress=[img_orig,initial,final_fundus,detection_on_orygin]
            self._image_progresses.append(image_progress)
            
            titles=['Original picture','Initial processing','Fundus mask','Applied on original']
            filepath="img/image-results/img"+str(i)+".jpg"
            display_results(image_progress, titles,filepath)
            
            for i,(result,applied) in enumerate(zip(self._result_masks,self._result_images)):
                display_picture(result,"img/image-results/manual-masks/img"+str(i)+".jpg")
                display_picture(applied,"img/image-results/manual-applied-"+str(i)+".jpg")
    
    def initial_processing(self, img):
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        merged = cv2.merge((cl,a,b))
        clahed_image = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
        clahed_image[:, :, 0] = 0
        clahed_image[:, :, 2] = 0
        img=img_as_float(rgb2gray(clahed_image))
        img=gaussian(img,sigma=3) 
        img=img**0.2

        mask=img_as_float(rgb2gray(self._official_mask[0]))
        img=img*mask

        return img

    def edge_detecting(self,img):
        img=sato(img)
        img = (img - np.min(img)) / (np.max(img) - np.min(img))

        img=mp.dilation(img,selem=mp.disk(6))
        img=gaussian(img,sigma=3) 
        img=mp.closing(img, selem=mp.disk(8))
        img=mp.erosion(img,selem=mp.disk(2))

        thresh = threshold_li(img, tolerance=0.0005)
        img = img > thresh

        mask=img_as_float(rgb2gray(self._official_mask[0]))
        mask=mp.erosion(mask, selem=mp.disk(5))
        img=img*mask

        return img

    def vessels_mask_creating(self,img):
        detection = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)

        for y in range(img.shape[0]):
            for x in range(img.shape[1]):
                if cv2.countNonZero(img[y, x]) > 0:
                    detection[y, x] = np.array([255, 255, 255], np.uint8)

        return detection

    def fundus_final_processing(self,img):
        img = np.array(img)
        img = 255*(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) > 5).astype('uint8')

        img=mp.erosion(img,selem=mp.disk(1))
        img=mp.closing(img, selem=mp.disk(15))
        img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)[1]

        return img

    def get_statistics(self):
        accuracy_list,sensitivity_list,precision_list,specificity_list,f_score_list,g_mean_list = calculate_statistics(self._result_masks, self._expected_array,"manual")
        table = pd.DataFrame({"Accuracy": accuracy_list,"Sensitivity": sensitivity_list,"Specificity": specificity_list, "Geometric mean":g_mean_list,"Precision": precision_list,"Recall":sensitivity_list,"F1":f_score_list})
        display(table)
        
    def get_images_array(self):
        return self._images_array
    
    def get_expected_array(self):
        return self._expected_array
    
    def get_official_mask(self):
        return self._official_mask
    
    def get_result_masks(self):
        return self._result_masks
    
    def get_result_images(self):
        return self._result_images
    
    def get_image_progresses(self):
        return self._image_progresses

### Class for classificator's fundus detecting

In [8]:
class ImageClassifier:
    
    def __init__(self, imageReader):
        self._images_array = imageReader.get_images_array()
        self._expected_array = imageReader.get_expected_array()
        self._official_mask = imageReader.get_official_mask()
        self._ImageProcessor = ImageProcessor(imageReader)
        
        self._model = None
        self._X, self._y = self.read_set_from_file()
        self._result_masks = []
        self._result_images = []
    
    def whole_fundus_finding_process(self):
        block_size = block_size_input.value
        self.create_data_set(block_size)
        self.train_model()
        self.find_fundus()
        self.get_statistics()
        
    def create_data_set(self, block_size):
        expected = self._expected_array[0]
        img = self._images_array[0]
        img = self._ImageProcessor.initial_processing(img)   
        
        parts,decisions = self.split_image(img,expected,block_size)

        data=[]
        for part,decision in zip(parts,decisions):
            one_part,_ = self.image_features_extract(part,decision)
            data.append(one_part)
        data_set = pd.DataFrame(data)
        print('Image based data frame:')
        display(data_set.head())
        data_set.to_csv('data/model_data.csv')
        
        return data_set
    
    def split_image(self,img,expect, block_size):
        windows=[]
        decisions = []
        for r in range(0,img.shape[0] - block_size, block_size):
            for c in range(0,img.shape[0] - block_size, block_size):
                window = img[r:r+block_size,c:c+block_size]
                exp_window = expect[r:r+block_size,c:c+block_size]
                center = np.amax(exp_window[exp_window.shape[0]//2,exp_window.shape[1]//2])
                if center>50:
                    center=1
                else:
                    center=0
                decisions.append(center)
                windows.append(window)

        return windows,decisions
            
    def image_features_extract(self, img, decision=None): 
        data={}
        data['variance'] = np.var(img)
        img = img_as_ubyte(rgb2gray(img))
        moments = cv2.moments(img)
        data={**data,**moments}
        hu_moments = cv2.HuMoments(moments)
        for i,hu in enumerate(hu_moments):
            name = "hu"+str(i)
            data[name]=hu[0]
        data['decision']=decision
        data_for_pic = [np.var(img),moments,hu_moments]
        return data, data_for_pic

    
    def read_set_from_file(self):
        if isfile('data/model_data.csv'):
            data_set = pd.read_csv('data/model_data.csv')
            data_set=data_set.drop("Unnamed: 0",axis=1)

            X = data_set.iloc[:, :-1].values
            y = data_set.iloc[:, -1].values
        else:
            display (Markdown('<span style="color: #ff0000">No image-data csv file</span>'))
            return
        
        return X,y
    
    def find_RandomizedSearchCV_best_params(self):
        # Number of trees in random forest
        n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
        max_features = ['auto', 'sqrt']
        max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
        max_depth.append(None)
        min_samples_split = [2, 5, 10]   # Minimum number of samples required to split a node
        min_samples_leaf = [1, 2, 4]     # Minimum number of samples required at each leaf node
        bootstrap = [True, False]        # Method of selecting samples for training each tree

        random_grid = {'n_estimators': n_estimators,
                       'max_features': max_features,
                       'max_depth': max_depth,
                       'min_samples_split': min_samples_split,
                       'min_samples_leaf': min_samples_leaf,
                       'bootstrap': bootstrap }

        print("Using the random grid to search for best parameters")
        print("Random Grid options: \n")
        pprint(random_grid)

        rf = RandomForestClassifier()
        rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        # n_iter controls the number of different combinations to try
        # cv is the number of folds to use for cross validation

        X = self._X
        y = self._y
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)

        rf_random.fit(X_train, y_train)
        best_random = rf_random.best_estimator_

        print("\nRandom best params:")
        pprint(rf_random.best_params_)


    def find_GridSearchCV_best_params(self):
        param_grid = {
            'bootstrap': [True],
            'max_depth': [1, 10, 20, 30],
            'max_features': [2, 3],
            'min_samples_leaf': [3, 4, 5],
            'min_samples_split': [2, 8, 12],
            'n_estimators': [100, 200, 400, 800]
        }  #grid parameters based on RandomizedSearchCV best params

        rf = RandomForestClassifier()
        grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                                  cv = 3, n_jobs = -1, verbose = 2)

        X = self._X
        y = self._y
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)

        grid_search.fit(X_train, y_train)
        best_grid = grid_search.best_estimator_

        print("\nGrid best params:")
        pprint(grid_search.best_params_)

      
    @jit
    def train_model(self,use_custom=False):
        X = self._X
        y = self._y

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)
        display_label_distribution(y_train,y_test)

        if use_custom==True:
            bootstrap,n_estimators,criterion,max_depth,max_features,min_samples_leaf,min_samples_split = get_input_model_params()
            rf_clf = RandomForestClassifier(bootstrap=bootstrap,n_estimators=n_estimators,criterion=criterion,max_depth=max_depth,max_features=max_features,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split)
        else:
            rf_clf = RandomForestClassifier(bootstrap=True,criterion='gini', 
                                            max_depth=30, max_features=3,
                                            max_leaf_nodes=None, max_samples=None,
                                            min_samples_leaf=4, min_samples_split=2,
                                            n_estimators=100, n_jobs=None)

        rf_clf.fit(X_train, y_train)
        rf_predictions = rf_clf.predict(X_train)
        rf_probs = rf_clf.predict_proba(X_train)[:, 1]
        fpr, tpr, _ = roc_curve(y_train,rf_probs)
        roc_auc = auc(fpr, tpr)

        print(classification_report(y_train, rf_predictions, labels=[0,1]))
        draw_roc_curve(tpr,fpr,roc_auc,'Receiver Operating Characteristic - train set')

        rf_predictions = rf_clf.predict(X_test)
        rf_probs = rf_clf.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test,rf_probs)
        roc_auc = auc(fpr, tpr)

        print(classification_report(y_test, rf_predictions, labels=[0,1]))
        draw_roc_curve(tpr,fpr,roc_auc,'Receiver Operating Characteristic - test set')

        self._model = rf_clf
    
    def find_fundus(self):
        for img in self._images_array:
            img = self._ImageProcessor.initial_processing(img)

            drawing_range={}
            drawing_range["x_start"]=0
            drawing_range["x_end"]=img.shape[0]
            drawing_range["y_start"]=0
            drawing_range["y_end"]=img.shape[1]
            result_mask = self.vessels_mask_creating(img,drawing_range)
            self._result_masks.append(result_mask)
        
        for i,result in enumerate(self._result_masks):
            display_picture(result,"img/image-results/model-masks/img"+str(i)+".jpg")
        
    @jit
    def vessels_mask_creating(self, img, drawing_range):
        block_size = block_size_input.value//2
        detection = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
        decisions=[]

        ###SPLITING IMAGE DRAWING INTO PARTS SO WON'T HAVE TO WAIT DAYS TO SEE RESULT
        x_start_point = drawing_range["x_start"]
        x_end_point = drawing_range["x_end"]
        y_start_point = drawing_range["y_start"]
        y_end_point = drawing_range["y_end"]

        print(x_end_point)
        maks_y = y_end_point-block_size-1

        for x in range(x_start_point + block_size, x_end_point - block_size):
            for y in range(y_start_point + block_size, y_end_point - block_size):
                window = img[x-block_size:x+block_size,y-block_size:y+block_size]

                _ ,data_set = self.image_features_extract(window)
                data_set = [[data_set[0]],list(data_set[1].values()),[item for sublist in data_set[2] for item in sublist]]
                data_set = [item for sublist in data_set for item in sublist]
                decision = self._model.predict([data_set])
                decisions.append(decision)
                if y == maks_y:
                    print(x,y)
                if decision!=0:
                    detection[x, y] = np.array([255, 255, 255], np.uint8)
        display_picture(detection)
        return detection   
    
    def get_statistics(self):
        accuracy_list,sensitivity_list,precision_list,specificity_list,f_score_list,g_mean_list = calculate_statistics(self._result_masks, self._expected_array,"model")
        table = pd.DataFrame({"Accuracy": accuracy_list,"Sensitivity": sensitivity_list,"Specificity": specificity_list, "Geometric mean":g_mean_list,"Precision": precision_list,"Recall":sensitivity_list,"F1":f_score_list})
        display(table)
        
    def get_images_array(self):
        return self._images_array
    
    def get_expected_array(self):
        return self._expected_array
    
    def get_official_mask(self):
        return self._official_mask
    
    def get_result_masks(self):
        return self._result_masks
    
    def get_result_images(self):
        return self._result_images

### On-click functions to handle interactions with the user

In [9]:
def on_upload_clicked(x):
    clear_output()
    display(choose_image_box)
    imageReader.upload_single_image(filename_input.value)
    
    for img in imageReader.get_images_array():
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        display_picture(img)

def on_update_clicked(x):
    filepath = filename_input.value
    imageReader.upload_single_image(filepath)
    print("Updated!")
    
def on_upload_many_clicked(x):
    clear_output()
    display(choose_image_box)
    
    path = filepath_input.value
    imageReader.upload_many_images(path)

def on_find_fundus_clicked(x):
    imageProcessor.find_fundus()

def on_check_stats_clicked(x, if_manual):
    if if_manual:
        result_masks = imageProcessor.get_result_masks()
        expected_array = imageProcessor.get_expected_array()
    else:
        result_masks = imageClassifier.get_result_masks()
        expected_array = imageClassifier.get_expected_array()
    accuracy_list,sensitivity_list,precision_list,specificity_list,f_score_list,g_mean_list = calculate_statistics(result_masks, expected_array)
    
    print("Accuracy is inappropriate for imbalanced classification because a high accuracy is achievable by a no skill model that only predicts the majority class.")
    
    print("Metrics that may be useful for imbalanced classification because they focus on one class are: sensitivity-specificity and precision-recall.")
    print("Sensitivity (recall) is  how well the positive class was predicted.")
    print('Specificity is  how well the negative class was predicted.')
    print('Sensitivity and specificity can be combined into a single score that balances both concerns, called the geometric mean.')
    
    print('Precision summarizes the fraction of examples assigned the positive class that belong to the positive class.')
    print("Precision and recall can be combined into a single score that seeks to balance both concerns, called the F-score.")
    
    table = pd.DataFrame({"Accuracy": accuracy_list,"Sensitivity": sensitivity_list,"Specificity": specificity_list, "Geometric mean":g_mean_list,"Precision": precision_list,"Recall":sensitivity_list,"F1":f_score_list})
    display(table)
    
def on_extract_image_data_clicked(x):
    block_size = block_size_input.value
    imageClassifier.create_data_set(block_size)

def on_train_model_clicked(x):
    use_custom=use_custom_input.value
    imageClassifier.train_model(use_custom)

def on_find_best_random_clicked(x):
    imageClassifier.find_RandomizedSearchCV_best_params()

def on_find_best_grid_clicked(x):
    imageClassifier.find_GridSearchCV_best_params()
    
def on_model_find_fundus_clicked(x):
    imageClassifier.find_fundus()
    
def on_final_process_finished_clicked(x):
    path = finished_path_input.value
    mask_imageReader = ImageReader()
    mask_imageReader.upload_many_images(path)
    
    orig_imageReader = ImageReader()
    orig_imageReader.upload_many_images("img/")
    for i,(mask,orig) in enumerate(zip(mask_imageReader.get_images_array(),orig_imageReader.get_images_array())):
        img=apply_pic_on_pic(orig,mask)
        display_picture(img,"img/image-results/model-applied-"+str(i)+".jpg")
    
    accuracy_list,sensitivity_list,precision_list,specificity_list,f_score_list,g_mean_list = calculate_statistics(mask_imageReader.get_images_array(), mask_imageReader.get_expected_array(), filename="models")
    table = pd.DataFrame({"Accuracy": accuracy_list,"Sensitivity": sensitivity_list,"Specificity": specificity_list, "Geometric mean":g_mean_list,"Precision": precision_list,"Recall":sensitivity_list,"F1":f_score_list})
    display(table)

### Settings for the buttons and other input options

In [10]:
horizontal_box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')

vertical_box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='100%')

style = {'description_width': 'initial'}

single_image_label= widgets.Label(value = 'Choose single image to process...')
files = [f for f in listdir("img/") if isfile(join("img/", f))]

filename_input = widgets.Dropdown(
    options=files,
    value='01_h.jpg',
    description='filename',
    disabled=False
)

upload_button = widgets.Button(description = "Upload image")
upload_button.style.button_color = 'lightpink'
upload_button.on_click(functools.partial(on_upload_clicked))

single_image_box = Box([single_image_label,filename_input,upload_button], layout = vertical_box_layout)

many_images_label= widgets.Label(value = '...or process the full directory')
filepath_input = widgets.Text(layout=Layout(width='350px'),description='directory',value='img/')
upload_many_button = widgets.Button(description = "Update images")
upload_many_button.style.button_color = 'lightpink'
upload_many_button.on_click(functools.partial(on_upload_many_clicked))


many_images_box = Box([many_images_label,filepath_input,upload_many_button], layout = vertical_box_layout)

items = [single_image_box,many_images_box]

choose_image_box = Box(children=items, layout=horizontal_box_layout)

find_fundus_button = widgets.Button(description="Find fundus")
find_fundus_button.style.button_color = 'lightpink'
find_fundus_button.on_click(functools.partial(on_find_fundus_clicked))

check_stats_button = widgets.Button(description="Check stats!")
check_stats_button.style.button_color = 'lightpink'
check_stats_button.on_click(functools.partial(on_check_stats_clicked, True))

sizes = [4,5,15,25,64]
warning_label= widgets.Label(value = 'Choose image to train your model in the upper single image choosing panel')
block_size_label= widgets.Label(value = 'Choose to what size blocks split images')
block_size_input = widgets.Dropdown(
    options=sizes,
    value=4,
    description='blocks size',
    disabled=False
)

extract_data_button = widgets.Button(description="Extract image data")
extract_data_button.style.button_color = 'lightpink'
extract_data_button.on_click(functools.partial(on_extract_image_data_clicked))

prepare_data_box = Box([warning_label,block_size_label,block_size_input,extract_data_button], layout = vertical_box_layout)

warning_random_label= widgets.Label(value = 'Warning! Finding best random parameters may take hours')
find_best_random_button = widgets.Button(description="Find best random params")
find_best_random_button.style.button_color = 'lightpink'
find_best_random_button.on_click(functools.partial(on_find_best_random_clicked))

find_params_left = Box([warning_random_label,find_best_random_button],layout=vertical_box_layout)

warning_grid_label= widgets.Label(value = 'Faster than random but still slow:')
find_best_grid_button = widgets.Button(description="Find best grid params")
find_best_grid_button.style.button_color = 'lightpink'
find_best_grid_button.on_click(functools.partial(on_find_best_grid_clicked))

find_params_right = Box([warning_grid_label,find_best_grid_button],layout=vertical_box_layout)

find_best_params_box = Box([find_params_left,find_params_right], layout = horizontal_box_layout)


set_model_params_label = widgets.Label(value='Enter model params or leave default')

bootstrap_input = widgets.Checkbox(
    value=True,
    description='Bootstrap',
    style = style
)

n_estimators_input = widgets.IntSlider(
    value=800,
    min=0,
    max=1000,
    step=100,
    description='Number of trees',
    style = style
)

criterion_input = widgets.Dropdown(
    options=['gini', 'entropy'],
    value='gini',
    description='Criterion',
    style = style
)

max_depth_input = widgets.IntSlider(
    value=20,
    min=0,
    max=100,
    step=10,
    description='Max depth',
    style = style
)

max_features_input = widgets.Dropdown(
    options=['auto', 'sqrt','2','3'],
    value='3',
    description='Max features',
)
min_samples_leaf_input = widgets.Dropdown(
    options=['1', '2','4'],
    value='4',
    description='Min samples leaf',
)

min_samples_split_input = widgets.Dropdown(
    options=['5','10','12','15'],
    value='12',
    description='Min samples split',
)

use_custom_input = widgets.Checkbox(
    value=False,
    description='Use this params',
    style = style
)

train_model_button = widgets.Button(description="Train best model!")
train_model_button.style.button_color = 'lightpink'
train_model_button.on_click(functools.partial(on_train_model_clicked))

left_params_box = Box([bootstrap_input,n_estimators_input,criterion_input,max_depth_input],layout = vertical_box_layout)
right_params_box = Box([max_features_input,min_samples_leaf_input,min_samples_split_input,use_custom_input],layout = vertical_box_layout)
params_box = Box([left_params_box,right_params_box],layout = horizontal_box_layout)
set_params_box = Box([set_model_params_label,params_box,train_model_button],layout = vertical_box_layout)

update_button = widgets.Button(description = "Update image")
update_button.style.button_color = 'lightpink'
update_button.on_click(functools.partial(on_update_clicked))

model_find_fundus=widgets.Button(description="Find fundus!")
model_find_fundus.style.button_color = 'lightpink'
model_find_fundus.on_click(functools.partial(on_model_find_fundus_clicked))
model_find_funudus_box = Box([single_image_label,filename_input,update_button,model_find_fundus], layout = vertical_box_layout)

check_model_stats_button = widgets.Button(description="Check model stats!")
check_model_stats_button.style.button_color = 'lightpink'
check_model_stats_button.on_click(functools.partial(on_check_stats_clicked, False))

warning_process_finished_label= widgets.Label(value = 'Enter models results mask path for applying and stats at the end:')
finished_path_input = widgets.Text(layout=Layout(width='350px'),description='directory',value='img/models-masks')

final_process_finished_button = widgets.Button(description="Apply and stats!")
final_process_finished_button.style.button_color = 'lightpink'
final_process_finished_button.on_click(functools.partial(on_final_process_finished_clicked))

final_masks_box = Box([warning_process_finished_label,finished_path_input,final_process_finished_button],layout=vertical_box_layout)